In [2]:
import os
import gc
import time
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(0)

import mxnet as mx
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
mx.random.seed(0)

from model import *
from utils import *

import warnings

warnings.filterwarnings("ignore", message="WARNING:gluonts.model.forecast:The mean prediction is not stored in the forecast data; the median is being returned instead. This behaviour may change in the future.")

In [ ]:
def train_models(epochs, learning_rate):    
    # save directory
    save_dir = f'../result/learning_rate_{learning_rate}-patience_15'
    os.makedirs(save_dir, exist_ok=True)
    
    for level_idx in range(8, 9):
        # level directory
        level_dir = os.path.join(save_dir, f'level_{level_idx}')
        os.makedirs(level_dir, exist_ok=True)

        highlight_print(f"\n========== Level {level_idx} ==========")
        print(f"Loading dataset...", end=' ')
        dataset_start = time.time()
        with open(os.path.join('../dataset', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
            dataset = pickle.load(f)
        print(f"{(time.time() - dataset_start)/60:.1f} minutes")

        print("Normalizing dataset...", end=' ')
        normalize_start = time.time()
        norm_train_dataset, norm_test_dataset, normalizer = normalize_dataset(
            dataset['train'], 
            dataset['test'],
        )
        with open(os.path.join(level_dir, 'normalizer.pkl'), 'wb') as f:
            pickle.dump(normalizer, f)
        print(f"{(time.time() - normalize_start)/60:.1f} minutes")

        estimators = create_estimators(
            level_idx=level_idx, 
            train_dataset=norm_train_dataset
        )
        for estimator_name, estimator in estimators.items():
            # estimator directory
            estimator_dir = os.path.join(level_dir, estimator_name)
            if any(existing_dir.startswith(estimator_name) for existing_dir in os.listdir(level_dir)):
                continue
            os.makedirs(estimator_dir, exist_ok=True)

            estimator_start = time.time()

            highlight_print(f"\n---------- {estimator_name} ----------")
            print("Start training...")
            train_start = time.time()
            estimator.trainer = EarlyStoppingTrainer(
                epochs=epochs,
                learning_rate=learning_rate,
                num_batches_per_epoch=get_optimal_num_batches(mx.context.num_gpus()),
                patience=15,
            )
            predictor = estimator.train(
                training_data=norm_train_dataset,
                validation_data=norm_test_dataset
            )
            predictor.serialize(Path(f"{level_dir}/{estimator_name}"))
            highlight_print(f"End training... {(time.time() - train_start)/60:.1f} minutes", color='green')

            print("Start predicting...")
            pred_start = time.time()
            train_forecasts_it, train_labels_it = make_evaluation_predictions(
                dataset=norm_train_dataset,
                predictor=predictor,
            )
            train_forecasts = list(train_forecasts_it)
            train_labels = list(train_labels_it)

            test_forecasts_it, test_labels_it = make_evaluation_predictions(
                dataset=norm_test_dataset,
                predictor=predictor,
            )
            test_forecasts = list(test_forecasts_it)
            test_labels = list(test_labels_it)
            highlight_print(f"End predicting... {(time.time() - pred_start)/60:.1f} minutes", color='green')

            print("Start saving...")
            pred_save_start = time.time()
            for i in range(len(train_forecasts)):
                train_forecasts[i] = normalizer.inverse_transform_forecast(train_forecasts[i])
            for i in range(len(test_forecasts)):
                test_forecasts[i] = normalizer.inverse_transform_forecast(test_forecasts[i])
            for i in range(len(train_labels)):
                train_labels[i] = normalizer.inverse_transform_labels(train_labels[i])
            for i in range(len(test_labels)):
                test_labels[i] = normalizer.inverse_transform_labels(test_labels[i])
            with open(f"{level_dir}/{estimator_name}/train_labels.pkl", "wb") as f:
                pickle.dump(train_labels, f)
            with open(f"{level_dir}/{estimator_name}/train_forecasts.pkl", "wb") as f:
                pickle.dump(train_forecasts, f)
            with open(f"{level_dir}/{estimator_name}/test_labels.pkl", "wb") as f:
                pickle.dump(test_labels, f)
            with open(f"{level_dir}/{estimator_name}/test_forecasts.pkl", "wb") as f:
                pickle.dump(test_forecasts, f)
            highlight_print(f"End saving... {(time.time() - pred_save_start)/60:.1f} minutes", color='green')

            # print("Start plotting...")
            # plot_start = time.time()
            # for i in range(len(test_forecasts)):
            #     plt.figure(figsize=(12, 6))
            #     plt.plot(test_labels[i][-100:].to_timestamp(), label="Actual")
            #     plt.plot(pd.Series(test_forecasts[i].quantile(0.5), index=test_forecasts[i].start_date.to_timestamp() + pd.to_timedelta(range(len(test_forecasts[0].quantile(0.5))), unit='D')), label="Forecast")
            #     plt.title(f'{test_forecasts[i].item_id}')
            #     plt.xlabel('Date')
            #     plt.ylabel('Sales')
            #     plt.legend(loc="upper right")
            #     plt.savefig(os.path.join(estimator_dir, f'series_{i + 1}.png'))
            #     plt.close()
            # highlight_print(f"End plotting... {(time.time() - plot_start)/60:.1f} minutes", color='green')
                
            print("Start evaluating...")
            eval_start = time.time()
            evaluator = Evaluator(quantiles=(0.5,), ignore_invalid_values=True)
            train_metrics_all_id, train_metrics_per_id = evaluator(train_labels, train_forecasts)
            test_metrics_all_id, test_metrics_per_id = evaluator(test_labels, test_forecasts)
            highlight_print(f"End evaluating... {(time.time() - eval_start)/60:.1f} minutes", color='green')

            print("Start saving...")
            eval_save_start = time.time()
            with open(f"{level_dir}/{estimator_name}/train_metrics_all_id.pkl", "wb") as f:
                pickle.dump(train_metrics_all_id, f)
            with open(f"{level_dir}/{estimator_name}/train_metrics_per_id.pkl", "wb") as f:
                pickle.dump(train_metrics_per_id, f)
            with open(f"{level_dir}/{estimator_name}/test_metrics_all_id.pkl", "wb") as f:
                pickle.dump(test_metrics_all_id, f)
            with open(f"{level_dir}/{estimator_name}/test_metrics_per_id.pkl", "wb") as f:
                pickle.dump(test_metrics_per_id, f)
            highlight_print(f"End saving... {(time.time() - eval_save_start)/60:.1f} minutes", color='green')

            highlight_print(f"\nTotal time: {(time.time() - estimator_start)/60:.1f} minutes", color='red')

            # rename estimator directory
            os.rename(estimator_dir, f"{estimator_dir}_{test_metrics_all_id['MASE']:.2f}")

            # reduce memory
            del estimator, predictor, train_forecasts, train_labels, test_forecasts, test_labels
            gc.collect()

train_models(epochs=500, learning_rate=1e-3)


========== Level 8 ==========
Loading dataset... 0.0 minutes
Normalizing dataset... 0.0 minutes
Directory for DeepAR already exists. Skipping...
Directory for MQRNN already exists. Skipping...
Directory for DeepState already exists. Skipping...
Directory for DeepFactor already exists. Skipping...
Directory for WaveNet already exists. Skipping...
Directory for MQCNN already exists. Skipping...
Directory for NBEATS already exists. Skipping...
Directory for Transformer already exists. Skipping...


In [ ]:
epochs=1
learning_rate=1e-3
level_idx=1

save_dir = f'../result/learning_rate_{learning_rate}'
os.makedirs(save_dir, exist_ok=True)

# level directory
level_dir = os.path.join(save_dir, f'level_{level_idx}')
os.makedirs(level_dir, exist_ok=True)

highlight_print(f"\n========== Level {level_idx} ==========")
print(f"Loading dataset...", end=' ')
dataset_start = time.time()
with open(os.path.join('../dataset', f'dataset_level_{level_idx}.pkl'), 'rb') as f:
    dataset = pickle.load(f)
print(f"{(time.time() - dataset_start)/60:.1f} minutes")

print("Normalizing dataset...", end=' ')
normalize_start = time.time()
norm_train_dataset, norm_test_dataset, normalizer = normalize_dataset(
    dataset['train'], 
    dataset['test'],
)
with open(os.path.join(level_dir, 'normalizer.pkl'), 'wb') as f:
    pickle.dump(normalizer, f)
print(f"{(time.time() - normalize_start)/60:.1f} minutes")

estimators = create_estimators(
    level_idx=level_idx, 
    train_dataset=norm_train_dataset
)
for estimator_name, estimator in estimators.items():
    # estimator directory
    estimator_dir = os.path.join(level_dir, estimator_name)
    os.makedirs(estimator_dir, exist_ok=True)

    estimator_start = time.time()

    highlight_print(f"\n---------- {estimator_name} ----------")
    print("Start training...")
    train_start = time.time()
    estimator.trainer = EarlyStoppingTrainer(
        epochs=epochs,
        learning_rate=learning_rate,
        num_batches_per_epoch=get_optimal_num_batches(mx.context.num_gpus()),
    )
    predictor = estimator.train(
        training_data=norm_train_dataset,
        validation_data=norm_test_dataset
    )
    predictor.serialize(Path(f"{level_dir}/{estimator_name}"))
    highlight_print(f"End training... {(time.time() - train_start)/60:.1f} minutes", color='green')

    print("Start predicting...")
    pred_start = time.time()
    train_forecasts_it, train_labels_it = make_evaluation_predictions(
        dataset=norm_train_dataset,
        predictor=predictor,
    )
    train_forecasts = list(train_forecasts_it)
    train_labels = list(train_labels_it)

    test_forecasts_it, test_labels_it = make_evaluation_predictions(
        dataset=norm_test_dataset,
        predictor=predictor,
    )
    test_forecasts = list(test_forecasts_it)
    test_labels = list(test_labels_it)
    highlight_print(f"End predicting... {(time.time() - pred_start)/60:.1f} minutes", color='green')

    print("Start saving...")
    pred_save_start = time.time()
    for i in range(len(train_forecasts)):
        train_forecasts[i] = normalizer.inverse_transform_forecast(train_forecasts[i])
    for i in range(len(test_forecasts)):
        test_forecasts[i] = normalizer.inverse_transform_forecast(test_forecasts[i])
    for i in range(len(train_labels)):
        train_labels[i] = normalizer.inverse_transform_labels(train_labels[i])
    for i in range(len(test_labels)):
        test_labels[i] = normalizer.inverse_transform_labels(test_labels[i])
    with open(f"{level_dir}/{estimator_name}/train_labels.pkl", "wb") as f:
        pickle.dump(train_labels, f)
    with open(f"{level_dir}/{estimator_name}/train_forecasts.pkl", "wb") as f:
        pickle.dump(train_forecasts, f)
    with open(f"{level_dir}/{estimator_name}/test_labels.pkl", "wb") as f:
        pickle.dump(test_labels, f)
    with open(f"{level_dir}/{estimator_name}/test_forecasts.pkl", "wb") as f:
        pickle.dump(test_forecasts, f)
    highlight_print(f"End saving... {(time.time() - pred_save_start)/60:.1f} minutes", color='green')

    print("Start plotting...")
    plot_start = time.time()
    for i in range(len(test_forecasts)):
        plt.figure(figsize=(12, 6))
        plt.plot(test_labels[i][-100:].to_timestamp(), label="Actual")
        plt.plot(pd.Series(test_forecasts[i].quantile(0.5), index=test_forecasts[i].start_date.to_timestamp() + pd.to_timedelta(range(len(test_forecasts[0].quantile(0.5))), unit='D')), label="Forecast")
        plt.title(f'{test_forecasts[i].item_id}', color='white')
        plt.xlabel('Date', color='white')
        plt.ylabel('Sales', color='white')
        plt.xticks(color='white')
        plt.yticks(color='white')
        plt.legend(loc="upper right")
        plt.savefig(os.path.join(estimator_dir, f'series_{i + 1}.png'))
        plt.close()
    highlight_print(f"End plotting... {(time.time() - plot_start)/60:.1f} minutes", color='green')
        
    print("Start evaluating...")
    eval_start = time.time()
    evaluator = Evaluator(quantiles=(0.5,), ignore_invalid_values=True)
    train_metrics_all_id, train_metrics_per_id = evaluator(train_labels, train_forecasts)
    test_metrics_all_id, test_metrics_per_id = evaluator(test_labels, test_forecasts)
    highlight_print(f"End evaluating... {(time.time() - eval_start)/60:.1f} minutes", color='green')

    print("Start saving...")
    eval_save_start = time.time()
    with open(f"{level_dir}/{estimator_name}/train_metrics_all_id.pkl", "wb") as f:
        pickle.dump(train_metrics_all_id, f)
    with open(f"{level_dir}/{estimator_name}/train_metrics_per_id.pkl", "wb") as f:
        pickle.dump(train_metrics_per_id, f)
    with open(f"{level_dir}/{estimator_name}/test_metrics_all_id.pkl", "wb") as f:
        pickle.dump(test_metrics_all_id, f)
    with open(f"{level_dir}/{estimator_name}/test_metrics_per_id.pkl", "wb") as f:
        pickle.dump(test_metrics_per_id, f)
    highlight_print(f"End saving... {(time.time() - eval_save_start)/60:.1f} minutes", color='green')

    highlight_print(f"\nTotal time: {(time.time() - estimator_start)/60:.1f} minutes", color='red')

    # rename estimator directory
    os.rename(estimator_dir, f"{estimator_dir}_{test_metrics_all_id['MASE']:.2f}")